In [4]:
# Import required libraries again due to environment reset
import os
import zipfile
import pandas as pd
import numpy as np

# Specify the paths for the zip file and the extraction directory
zip_path = '/home1/mkato/hdd_data/data/0-0-raw_vcf/stats/stats.zip'
unzip_dir = '/home1/mkato/hdd_data/data/0-0-raw_vcf/stats/unzip'

In [5]:
# Function to parse tables
def parse_table_new(lines, header_format):
    table = []
    for line in lines:
        if line.startswith("#"):
            header = line[1:].split("\t")  # Remove "#" and split by tabs
            header = [h.split(" ")[0] for h in header]  # Keep only the first word of each header element
        else:
            values = line.split("\t")
            row = dict(zip(header, values))
            table.append(row)
    return table

In [6]:
# Column renaming dictionaries
qual_columns = {
    '[2]id': 'id',
    '[3]Quality': 'Quality',
    '[4]number': 'number_of_variants',
    '[5]number': 'number_of_transitions',
    '[6]number': 'number_of_transversions',
    '[7]number': 'number_of_indels'
}

dp_columns = {
    '[2]id': 'id',
    '[3]bin': 'bin',
    '[4]number': 'number_of_genotypes',
    '[5]fraction': 'fraction_of_genotypes',
    '[6]number': 'number_of_sites',
    '[7]fraction': 'fraction_of_sites'
}

In [7]:
# Unzipping the files again
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# List of unzipped files
unzipped_files = os.listdir(unzip_dir)

# Extracting QUAL and DP tables from the stats files again
qual_data = {}
dp_data = {}

for file_name in unzipped_files:
    individual = file_name.split('_')[0]
    stats_path = os.path.join(unzip_dir, file_name)
    
    qual_section = []
    dp_section = []
    in_qual = False
    in_dp = False
    
    with open(stats_path, 'r') as f:
        for line in f:
            cols = line.strip().split("\t")
            if cols[0].startswith("# DP"):
                in_qual = False
                in_dp = True
                dp_section.append(line.strip())
                continue
            if in_dp:
                dp_section.append(line.strip())

    dp_data[individual] = parse_table_new(dp_section, header_format="DP")

print("parse table done")

parse table done


In [10]:
# Convert the extracted data to Pandas DataFrames again
dp_dfs = {}

for individual, data in dp_data.items():
    dp_df = pd.DataFrame(data)
    dp_df.rename(columns=dp_columns, inplace=True)
    dp_df.drop(columns=[''], inplace=True)
    dp_df = dp_df.apply(pd.to_numeric, errors='ignore')
    dp_dfs[individual] = dp_df

print("regenerate DataFrame done")

# Function to calculate the average depth for each sample
def calculate_average_depth(data):
    avg_depths = {}
    for sample, df in data.items():
        # Convert the 'bin' to numeric, treating '>500' as 500
        df['bin_numeric'] = pd.to_numeric(df['bin'], errors='coerce').fillna(500)
        avg_depth = np.average(df['bin_numeric'], weights=df['number_of_sites'])
        avg_depths[sample] = avg_depth
    return avg_depths

avg_depths = calculate_average_depth(dp_dfs)
print("calculate done")
print(avg_depths)


regenerate DataFrame done
calculate done
{'T5': 48.75800554480932, 'F23': 43.226838270400314, 'FM020': 44.18137783277203, 'I4': 32.79694664347636}
